In [1]:
import warnings
warnings.filterwarnings("ignore")

# 1. Using Selenium to download 10-K financial dataset for relevant years

In [2]:
import time
import zipfile
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import glob

In [33]:
#Setting up browser and broswer functions
dwpath = r'/Users/diegoaub/Desktop/10-K Files'
drpath = r'/Users/diegoaub/.local/geckodriver'

#Time Range
year_max = 2018
year_min = 2017

def get_10K_files(drpath,dwpath,year_max,year_min):
    
    options = Options()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", dwpath)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
    browser = webdriver.Firefox(drpath, options=options)

    webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')

    wait = WebDriverWait(browser, 10)

    def click(element): 
        wait.until(ec.element_to_be_clickable((By.XPATH, element))).click()   

    def get(element):
        return browser.find_element(By.XPATH,element)

    #Going to SEC data sets page
    browser.get(f"https://www.sec.gov/dera/data/financial-statement-data-sets")

    #Clicking on downloads for wanted financial year
    dw_df = pd.read_html(get('//*[@id="block-secgov-content"]/article/div[1]/div[2]/div[4]/div/div/table').get_attribute("outerHTML"))[0]

    years = []
    for x in range(year_min,year_max+1):
        years.append(str(x))

    index_df = dw_df[dw_df['File'].str.contains('|'.join(years))].index + 1
    for index in index_df:
        time.sleep(1)
        click(f'//*[@id="block-secgov-content"]/article/div[1]/div[2]/div[4]/div/div/table/tbody/tr[{index}]')

    time.sleep(10)

    #Downloading and openning zip files   
    zips = [file for file in listdir(dwpath) if ((isfile(join(dwpath, file)))&('zip' in file))]
    extracts = [file for file in listdir(dwpath) if ((isfile(join(dwpath, file)))&('zip' not in file)&(file[0] != '.'))]
    for file in zips:
        with zipfile.ZipFile(dwpath+'//'+file, 'r') as zip_ref:
            extracts = zip_ref.namelist()
            zip_ref.extractall(dwpath)
            for extract in extracts:
                os.rename(dwpath+'//'+extract,dwpath+f'//{file[:-4]} {extract}')

    #Setting up DataFrames for files with company information
    nums = [file for file in listdir(dwpath) if 'num' in file]
    subs = [file for file in listdir(dwpath) if 'sub' in file]     
    pre = [file for file in listdir(dwpath) if 'pre' in file]              
    num_df = pd.DataFrame()
    sub_df = pd.DataFrame()
    pre_df = pd.DataFrame()
    for i in range(len(nums)):
        num_d = pd.read_csv(dwpath+'//'+nums[i],sep='	')
        num_df = num_df.append(num_d)
        sub_d = pd.read_csv(dwpath+'//'+subs[i],sep='	')
        sub_df = sub_df.append(sub_d)
        pre_d = pd.read_csv(dwpath+'//'+pre[i],sep='	')
        pre_df = pre_df.append(pre_d)

    #Waiting to ensure appending has time to finish before filtering happens
    time.sleep(5)

    #Filtering DataFrame
    sub_df = sub_df[(sub_df['form']=='10-K')]
    sub_df = sub_df.sort_values(by='adsh').reset_index().drop('index',axis=1)
    
    #Removing Downloaded Files
    files = glob.glob(dwpath+r'/*')
    for file in files:
        os.remove(file)
        
    return sub_df,num_df

In [34]:
sub_df,num_df = get_10K_files(drpath,dwpath,year_max,year_min)

In [35]:
sub_df

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0000002178-17-000019,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,...,20161231,2016.0,FY,20170331,2017-03-31 12:28:00.0,0,1,ae-20161231.xml,1,NaN
1,0000002178-18-000009,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,...,20171231,2017.0,FY,20180312,2018-03-12 17:29:00.0,0,1,ae-20171231.xml,1,NaN
2,0000002488-17-000043,2488,ADVANCED MICRO DEVICES INC,3674.0,US,CA,SUNNYVALE,94085,ONE AMD PL,MS 68,...,20161231,2016.0,FY,20170221,2017-02-21 16:52:00.0,0,1,amd-20161231.xml,1,NaN
3,0000002488-18-000042,2488,ADVANCED MICRO DEVICES INC,3674.0,US,CA,SANTA CLARA,95054,2485 AUGUSTINE DRIVE,NaN,...,20171231,2017.0,FY,20180227,2018-02-27 16:19:00.0,0,1,amd-20171230.xml,1,NaN
4,0000002969-17-000039,2969,AIR PRODUCTS & CHEMICALS INC /DE/,2810.0,US,PA,ALLENTOWN,18195-1501,7201 HAMILTON BLVD,NaN,...,20170930,2017.0,FY,20171116,2017-11-16 08:20:00.0,0,1,apd-20170930.xml,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11959,0001728688-18-000027,1728688,"I3 VERTICALS, INC.",7389.0,US,TN,NASHVILLE,37215,40 BURTON HILLS BLVD.,SUITE 415,...,20180930,2018.0,FY,20181207,2018-12-07 13:28:00.0,0,1,iiiv-20180930_htm.xml,1,NaN
11960,0001728949-18-000095,804328,QUALCOMM INC/DE,3663.0,US,CA,SAN DIEGO,92121,5775 MOREHOUSE DR,NaN,...,20180930,2018.0,FY,20181107,2018-11-07 16:16:00.0,0,1,qcom-20180930.xml,1,NaN
11961,0001730168-18-000084,1730168,BROADCOM INC.,3674.0,US,CA,SAN JOSE,95131,1320 RIDDER PARK DRIVE,NaN,...,20181031,2018.0,FY,20181221,2018-12-21 17:18:00.0,0,1,avgo-20181104.xml,1,NaN
11962,0001731122-18-000094,1527728,ENOCHIAN BIOSCIENCES INC,2834.0,US,CA,SUITE 906 LOS ANGELES,90067,CENTURY CITY MEDICAL PLAZA,2080 CENTURY CITY EAST,...,20180630,2018.0,FY,20181001,2018-09-28 21:58:00.0,0,1,enob-20180630.xml,1,NaN


In [36]:
num_df

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001654954-17-000061,AccountsPayableCurrent,us-gaap/2015,NaN,20141231,0,USD,7.266270e+05,NaN
1,0001654954-17-000061,AccountsPayableCurrent,us-gaap/2015,NaN,20151231,0,USD,1.423503e+06,NaN
2,0001654954-17-000061,AccountsPayableCurrent,us-gaap/2015,NaN,20160930,0,USD,1.851778e+06,NaN
3,0000940944-17-000002,AccountsPayableCurrent,us-gaap/2015,NaN,20160531,0,USD,2.419000e+08,NaN
4,0000940944-17-000002,AccountsPayableCurrent,us-gaap/2015,NaN,20161130,0,USD,2.496000e+08,NaN
...,...,...,...,...,...,...,...,...,...
2345131,0001390777-18-000083,PledgedFinancialInstrumentsNotSeparatelyReport...,0001390777-18-000083,NaN,20180331,0,USD,4.000000e+09,NaN
2345132,0001390777-18-000083,ProceedsFromMaturitiesAndCallsOfHeldToMaturity...,0001390777-18-000083,NaN,20170331,1,USD,2.469000e+09,NaN
2345133,0001390777-18-000083,ProceedsFromMaturitiesAndCallsOfHeldToMaturity...,0001390777-18-000083,NaN,20180331,1,USD,3.468000e+09,NaN
2345134,0001390777-18-000083,QualifiedAffordableHousingProjectInvestmentsCo...,0001390777-18-000083,NaN,20180331,0,USD,1.000000e+06,NaN


# 2. Using Beautiful Soup to fetch S&P 500 data for relevant years

In [37]:
import bs4 as bs
import urllib.request

In [58]:
#URL used to grab S&P 500 Composition at relevant time
url = "https://web.archive.org/web/20170820205408/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [59]:
def grab_SP500_data(url,sp_column_name):
    #Grabbing S&P 500 Table with CIK cods
    html = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(html,'lxml')
    table = soup.find_all("table",{'class': ['wikitable', 'sortable']})
    sp500_df = pd.read_html(str(table))[0]

    #Removing companies added in 2023
    sp500_df['Date first added'] = pd.to_datetime(sp500_df['Date first added'], format='%Y-%m-%d',errors='coerce')
    #sp500_df = sp500_df[sp500_df['Date first added'].dt.year < year_max]
    sp500_df[sp_column_name] = sp500_df[sp_column_name].str.replace('.','-')
    
    return sp500_df

In [60]:
#Grabbing S&P 500 data
sp500_df = grab_SP500_data(url,'Ticker symbol')

#Renaming first column (For ease of use)
cols = list(sp500_df.columns)
cols[0] = 'Symbol'
sp500_df.columns = cols

In [61]:
sp500_df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaT,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877
...,...,...,...,...,...,...,...,...
500,XYL,Xylem Inc.,reports,Industrials,Industrial Machinery,"White Plains, New York",2011-11-01,1524472
501,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061
502,ZBH,Zimmer Biomet Holdings,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",NaT,109380


# 3. HTTPS requests to calculate historical average S&P stocks price

In [62]:
from datetime import datetime

In [63]:
#Grabbing average close for 2022 and 2023
def get_close_avgs(year_min,year_max):
    period1a = int(time.mktime(datetime(year_min, 1, 1, 23, 59).timetuple()))
    period2a = int(time.mktime(datetime(year_min, 12, 31, 23, 59).timetuple()))
    period1b = int(time.mktime(datetime(year_max, 1, 1, 23, 59).timetuple()))
    period2b = int(time.mktime(datetime(year_max, 12, 31, 23, 59).timetuple()))
    interval = '1mo'

    avgmin = []
    avgmax = []

    failed = 0 
    successful_tickers = []

    #Query is made to yahoo finance based on the ticker and period
    for ticker in sp500_df['Symbol']:
        query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1a}&period2={period2a}&interval={interval}&events=history&includeAdjustedClose=true'
        query_string2 = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1b}&period2={period2b}&interval={interval}&events=history&includeAdjustedClose=true'
        try:
            hist_min = pd.read_csv(query_string)
            hist_max = pd.read_csv(query_string2)
            successful_tickers.append(ticker)
            avgmin.append(hist_min['Close'].mean())
            avgmax.append(hist_max['Close'].mean())
        except:
            failed += 1

    #Putting data together
    closes = pd.DataFrame(data=[successful_tickers,avgmin,avgmax]).transpose()
    closes.columns = ['Symbol',f'{year_min} Year Avg',f'{year_max} Year Avg']

    print(f'Could not retrieve {failed}/{len(sp500_df)}')
    
    return closes,successful_tickers

In [64]:
#Grabbing and calculating average prices
closes,successful_tickers = get_close_avgs(year_min,year_max)

Could not retrieve 84/505


In [66]:
closes

,Symbol,2017 Year Avg,2018 Year Avg
0,MMM,207.090001,209.711666
1,ABT,49.193333,65.343334
2,ABBV,75.877501,96.495001
3,ACN,130.3175,158.957499
4,AYI,185.846666,134.148333
...,...,...,...
416,XYL,57.677499,72.671668
417,YUM,72.695,85.7825
418,ZBH,116.25,113.711974
419,ZION,44.675,51.2975


# 4. Using CIKs to scrape data from 10-K financial dataset

In [280]:
def fetch_fundamentals(successful_tickers,sub_df,num_df,year_min):
    cumulative_fundamentals = pd.DataFrame()
    failed = 0
    for x in range(len(successful_tickers)):

        CIK = sp500_df[sp500_df['Symbol']==successful_tickers[x]]['CIK'].iloc[0]

        #Setting up dataframes to collect data and tags used to retrieve data
        revenue_df = pd.DataFrame() 
        revenue_names = ['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax','RevenueFromContractWithCustomerIncludingAssessedTax','SalesRevenueNet','SalesRevenueGoodsNet','AssetManagementFees1','RealEstateRevenueNet','OilAndGasRevenue','SalesRevenueServicesNet']

        income_df = pd.DataFrame() 
        net_income_names = ['ProfitLoss', 'NetIncomeLossIncludingPortionAttributableToNonredeemableNoncontrollingInterest', 'NetIncomeLoss']

        cash_df = pd.DataFrame() 
        cash_names = ['CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents','CashAndCashEquivalentsAtCarryingValue','CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations','NonRecourseDebt']

        int_df = pd.DataFrame() 
        int_names = ['InterestExpense','InterestExpenseDebt','InterestExpenseDeposits','InterestExpenseOther','']

        pref_df = pd.DataFrame() 
        pref_names = ['PaymentsOfDividendsPreferredStockAndPreferenceStock','DividendsPreferredStock','DividendsPreferredStockCash','DividendsPreferredStockStock']

        asset_df = pd.DataFrame() 
        asset_names = ['Assets']

        tdebt_df = pd.DataFrame() 
        tdebt_names = ['LongTermDebt','DebtInstrumentCarryingAmount','LongTermDebtAndCapitalLeaseObligationsGross','DebtLongtermAndShorttermCombinedAmount','LongTermDebtAndCapitalLeaseObligations','DebtAndCapitalLeaseObligations','DebtInstrumentFaceAmount','LongTermDebtNoncurrent','LongTermDebtFairValue']

        lialcur_df = pd.DataFrame() 
        lialcur_names = ['LiabilitiesCurrent']

        assetcur_df = pd.DataFrame() 
        assetcur_names = ['AssetsCurrent']

        equity_df = pd.DataFrame() 
        equity_names = ['StockholdersEquity']

        debtcurrent_df = pd.DataFrame() 
        debtcurrent_names = ['LongTermDebtCurrent','DebtCurrent','ConvertibleDebtCurrent','SecuredDebtCurrent','OtherLongTermDebtCurrent','UnsecuredDebtCurrent','SubordinatedDebtCurrent']  

        #Joining CSV files
        sub_df_K = sub_df[(sub_df['form']=='10-K')]
        sub_df_K = sub_df_K.sort_values(by='adsh').reset_index().drop('index',axis=1)
        result = sub_df_K[sub_df_K['cik']==CIK]
        try:
            n = result.iloc[-1][0]
            narrowed_num = num_df[num_df['adsh']==n]
            narrowed_num['ddate'] = pd.to_datetime(narrowed_num['ddate'], format='%Y%m%d')
            narrowed_num = narrowed_num[narrowed_num['ddate'].dt.year==year_min]

            #Attempting to retrieve data
            for name in revenue_names:
                revenue_df = revenue_df.append(narrowed_num[narrowed_num['tag']==name])
            revenue_df = revenue_df[revenue_df['qtrs'] == 4]
            revenue_df.sort_values(by=['ddate','value'],ascending=False)

            for name in net_income_names:
                    income_df = income_df.append(narrowed_num[narrowed_num['tag']==name])
            income_df = income_df[income_df['qtrs'] == 4]
            income_df = income_df.sort_values(by=['ddate','value'],ascending=False)

            for name in cash_names:
                    cash_df = cash_df.append(narrowed_num[narrowed_num['tag']==name])
            cash_df = cash_df[cash_df['qtrs'] == 0]
            cash_df = cash_df.sort_values(by=['ddate','value'],ascending=False)

            for name in int_names:
                    int_df = int_df.append(narrowed_num[narrowed_num['tag']==name])
            int_df = int_df.sort_values(by=['ddate','value'],ascending=False)

            for name in pref_names:
                    pref_df = pref_df.append(narrowed_num[narrowed_num['tag']==name])
            pref_df = pref_df.sort_values(by=['ddate','value'],ascending=False)

            for name in asset_names:
                    asset_df = asset_df.append(narrowed_num[narrowed_num['tag']==name])
            asset_df = asset_df.sort_values(by=['ddate','value'],ascending=False)

            for name in tdebt_names:
                    tdebt_df = tdebt_df.append(narrowed_num[narrowed_num['tag']==name])
            tdebt_df = tdebt_df.sort_values(by=['ddate','value'],ascending=False)

            for name in lialcur_names:
                    lialcur_df = lialcur_df.append(narrowed_num[narrowed_num['tag']==name])
            lialcur_df = lialcur_df.sort_values(by=['ddate','value'],ascending=False)

            for name in assetcur_names:
                    assetcur_df = assetcur_df.append(narrowed_num[narrowed_num['tag']==name])
            assetcur_df = assetcur_df.sort_values(by=['ddate','value'],ascending=False)

            for name in equity_names:
                    equity_df = equity_df.append(narrowed_num[narrowed_num['tag']==name])
            equity_df = equity_df.sort_values(by=['ddate','value'],ascending=False)

            for name in debtcurrent_names:
                    debtcurrent_df = debtcurrent_df.append(narrowed_num[narrowed_num['tag']==name])
            debtcurrent_df = debtcurrent_df.sort_values(by=['ddate','value'],ascending=False)

            #If empty, add dummy data
            if len(revenue_df) == 0 :
                revenue_df.loc[0] = 0
            if len(income_df) == 0 :
                income_df.loc[0] = 0
            if len(cash_df) == 0 :
                cash_df.loc[0] = 0
            if len(int_df) == 0 :
                int_df.loc[0] = 0
            if len(pref_df) == 0 :
                pref_df.loc[0] = 0
            if len(asset_df) == 0 :
                asset_df.loc[0] = 0
            if len(tdebt_df) == 0 :
                tdebt_df.loc[0] = 0
            if len(lialcur_df) == 0 :
                lialcur_df.loc[0] = 0
            if len(assetcur_df) == 0 :
                assetcur_df.loc[0] = 0
            if len(equity_df) == 0 :
                equity_df.loc[0] = 0
            if len(debtcurrent_df) == 0 :
                debtcurrent_df.loc[0] = 0

            #Sorting
            revenue_df = revenue_df.sort_values('value',ascending=False)
            income_df = income_df.sort_values('value',ascending=False)
            cash_df = cash_df.sort_values('value',ascending=False)
            int_df = int_df.sort_values('value',ascending=False)
            pref_df = pref_df.sort_values('value',ascending=False)
            asset_df = asset_df.sort_values('value',ascending=False)
            tdebt_df = tdebt_df.sort_values('value',ascending=False)
            lialcur_df = lialcur_df.sort_values('value',ascending=False)
            assetcur_df = assetcur_df.sort_values('value',ascending=False)
            equity_df = equity_df.sort_values('value',ascending=False)
            debtcurrent_df = debtcurrent_df.sort_values('value',ascending=False)

            #Putting in one dataframe
            fundamentals_df = pd.DataFrame([successful_tickers[x],CIK,revenue_df['value'].iloc[0],income_df['value'].iloc[0],cash_df['value'].iloc[0],int_df['value'].iloc[0],pref_df['value'].iloc[0],asset_df['value'].iloc[0],tdebt_df['value'].iloc[0],lialcur_df['value'].iloc[0],assetcur_df['value'].iloc[0],equity_df['value'].iloc[0],debtcurrent_df['value'].iloc[0]])
            fundamentals_df = fundamentals_df.transpose()
            fundamentals_df.columns = ['Symbol','CIK','Revenue','Net Income','Cash','Interest Expense','Preffered Shares','Assets','Total Debt','Current Liabilities','Current Assets','Equity','Current Debt']

            #Appending to cumulative dataframe
            cumulative_fundamentals = cumulative_fundamentals.append(fundamentals_df)
        except:
            failed += 1
            pass

    print(f'Could not process {failed}/{len(successful_tickers)}')
    
    return cumulative_fundamentals

In [271]:
#Scraping data from 10-K dataset
cumulative_fundamentals = fetch_fundamentals(successful_tickers,sub_df,num_df,year_min)

Could not process 2/421


In [272]:
cumulative_fundamentals

,Symbol,CIK,Revenue,Net Income,Cash,Interest Expense,Preffered Shares,Assets,Total Debt,Current Liabilities,Current Assets,Equity,Current Debt
0,MMM,66740,31657000000.0,4869000000.0,3053000000.0,322000000.0,0.0,37987000000.0,13198000000.0,7687000000.0,14277000000.0,11563000000.0,1853000000.0
0,ABT,1800,27390000000.0,477000000.0,9407000000.0,904000000.0,0.0,76250000000.0,27718000000.0,8912000000.0,20147000000.0,30897000000.0,508000000.0
0,ABBV,1551152,28216000000.0,5309000000.0,9303000000.0,1150000000.0,0.0,70786000000.0,37612000000.0,16641000000.0,21223000000.0,5097000000.0,0.0
0,ACN,1467373,36765478000.0,3445149000.0,4126860000.0,15545000.0,0.0,22689890000.0,25070000.0,9824279000.0,12097289000.0,8949477000.0,2907000.0
0,AYI,1144215,3505100000.0,321700000.0,311100000.0,34100000.0,0.0,2899600000.0,356900000.0,600900000.0,1245600000.0,1665600000.0,400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,XYL,1524472,4707000000.0,331000000.0,414000000.0,82000000.0,0.0,6860000000.0,2200000000.0,1100000000.0,2071000000.0,2503000000.0,0.0
0,YUM,1041061,0.0,1340000000.0,0.0,473000000.0,0.0,5311000000.0,9796000000.0,1512000000.0,2507000000.0,0.0,386000000.0
0,ZBH,1136869,7824100000.0,1813800000.0,524400000.0,327500000.0,0.0,25964500000.0,8917500000.0,3020200000.0,4515300000.0,11735800000.0,1225000000.0
0,ZION,109380,0.0,592000000.0,0.0,127000000.0,40000000.0,66288000000.0,383000000.0,0.0,0.0,0.0,0.0


# 5. Cleaning and merging collected data

In [277]:
#Merging DataFrames
data = pd.merge(closes,cumulative_fundamentals,on='Symbol')
data = data[(data['Revenue'] != 0)&(data['Net Income'] != 0)]

In [328]:
data

,Symbol,2017 Year Avg,2018 Year Avg,CIK,Revenue,Net Income,Cash,Interest Expense,Preffered Shares,Assets,Total Debt,Current Liabilities,Current Assets,Equity,Current Debt
0,MMM,207.090001,209.711666,66740,31657000000.0,4869000000.0,3053000000.0,322000000.0,0.0,37987000000.0,13198000000.0,7687000000.0,14277000000.0,11563000000.0,1853000000.0
1,ABT,49.193333,65.343334,1800,27390000000.0,477000000.0,9407000000.0,904000000.0,0.0,76250000000.0,27718000000.0,8912000000.0,20147000000.0,30897000000.0,508000000.0
2,ABBV,75.877501,96.495001,1551152,28216000000.0,5309000000.0,9303000000.0,1150000000.0,0.0,70786000000.0,37612000000.0,16641000000.0,21223000000.0,5097000000.0,0.0
3,ACN,130.3175,158.957499,1467373,36765478000.0,3445149000.0,4126860000.0,15545000.0,0.0,22689890000.0,25070000.0,9824279000.0,12097289000.0,8949477000.0,2907000.0
4,AYI,185.846666,134.148333,1144215,3505100000.0,321700000.0,311100000.0,34100000.0,0.0,2899600000.0,356900000.0,600900000.0,1245600000.0,1665600000.0,400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,XEL,46.8075,47.1075,72903,11404000000.0,1148000000.0,83000000.0,663000000.0,0.0,43030000000.0,14520000000.0,4088000000.0,2973000000.0,11455000000.0,151000000.0
413,XRX,29.83,27.601667,108772,10265000000.0,207000000.0,1293000000.0,252000000.0,17000000.0,15946000000.0,5573000000.0,2741000000.0,5230000000.0,5256000000.0,282000000.0
414,XYL,57.677499,72.671668,1524472,4707000000.0,331000000.0,414000000.0,82000000.0,0.0,6860000000.0,2200000000.0,1100000000.0,2071000000.0,2503000000.0,0.0
416,ZBH,116.25,113.711974,1136869,7824100000.0,1813800000.0,524400000.0,327500000.0,0.0,25964500000.0,8917500000.0,3020200000.0,4515300000.0,11735800000.0,1225000000.0


# 6. Creating final dataframe with ratios

In [20]:
import numpy as np 

In [617]:
#Creating Ratios

def calculate_ratios(data,year_max,year_min):
    ratios = pd.DataFrame()
    ratios['Symbol'] = data['Symbol']
    
    ratios['Revenue Increase'] = data[f'{year_max} Year Avg']/data[f'{year_min} Year Avg']
        
    ratios['Cash Ratio'] = data['Cash']/data['Current Liabilities']
    
    ratios['Current Ratio'] = data['Current Assets']/data['Current Liabilities']

    ratios['Interest to Income'] = data['Interest Expense']/data['Net Income']

    ratios['Debt to Income'] = data['Current Debt']/data['Net Income']

    ratios['Net income Margin'] = data['Net Income']/data['Revenue']

    ratios['ROA'] = data['Net Income']/data['Assets']

    ratios['ROE'] = data['Net Income']/data['Equity']

    ratios['ROE'] = data['Net Income']/data['Equity']

    ratios['Debt-to-assets'] = data['Total Debt']/data['Assets']

    ratios['Debt-to-equity'] = data['Total Debt']/data['Equity']

    ratios = ratios.replace(np.inf, np.nan).dropna()
    
    ratios = ratios.replace(0,np.nan).dropna(how='any',axis=0)
        
    ratios['Hist Revenue Increase'] =  np.where(ratios['Revenue Increase']>=1,1,0)

    return ratios

In [623]:
ratios = calculate_ratios(data,year_max,year_min)

# 8. Getting Validation Data (Basically Steps 1-6 Again)

In [320]:
#Step 1
year_max = 2023
year_min = 2022
sub_df_val,num_df_val = get_10K_files(drpath,dwpath,year_max,year_min)

#Step 2
url = "https://web.archive.org/web/20221221170536/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp_column_name = 'Symbol'
sp500_df_val = grab_SP500_data(url,sp_column_name)

#Step 3
closes_val,successful_tickers_val = get_close_avgs(year_min,year_max)

#Step 4
cumulative_fundamentals_val = fetch_fundamentals(successful_tickers_val,sub_df_val,num_df_val,year_min)

Could not retrieve 78/505
Could not process 11/427


In [353]:
closes_val

,Symbol,2022 Year Avg,2023 Year Avg
0,MMM,136.353334,103.223333
1,ABT,110.884167,104.584167
2,ABBV,147.9625,147.414166
3,ACN,298.915827,304.464165
4,AYI,175.500832,173.270835
...,...,...,...
422,XYL,93.166666,104.030833
423,YUM,119.448334,130.552502
424,ZBH,116.587693,125.308333
425,ZION,56.84,36.225833


In [324]:
data_val

,Symbol,2022 Year Avg,2023 Year Avg,CIK,Revenue,Net Income,Cash,Interest Expense,Preffered Shares,Assets,Total Debt,Current Liabilities,Current Assets,Equity,Current Debt
0,MMM,136.353334,103.223333,66740,34229000000.0,5791000000.0,3655000000.0,462000000.0,0.0,46455000000.0,15939000000.0,9523000000.0,14688000000.0,14722000000.0,1938000000.0
1,ABT,110.884167,104.584167,1800,43653000000.0,6933000000.0,9882000000.0,558000000.0,0.0,74438000000.0,16773000000.0,15489000000.0,25224000000.0,36686000000.0,2251000000.0
2,ABBV,147.9625,147.414166,1551152,58054000000.0,11845000000.0,9201000000.0,2230000000.0,0.0,138805000000.0,63270000000.0,29538000000.0,28463000000.0,17254000000.0,0.0
3,ACN,298.915827,304.464165,1467373,61594305000.0,6877169000.0,7889833000.0,47320000.0,0.0,47263390000.0,55068000.0,17523496000.0,21610871000.0,22106097000.0,9175000.0
4,AYI,175.500832,173.270835,1144215,4006100000.0,384000000.0,223200000.0,27000000.0,0.0,3480200000.0,513000000.0,733600000.0,1466000000.0,1911800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,WYNN,71.866667,100.715834,1174922,3756825000.0,-423856000.0,3782990000.0,650885000.0,0.0,13415100000.0,12162973000.0,1811423000.0,4029587000.0,-750838000.0,547543000.0
411,XYL,93.166666,104.030833,1524472,5522000000.0,355000000.0,944000000.0,50000000.0,0.0,7952000000.0,1880000000.0,1590000000.0,3012000000.0,3494000000.0,0.0
412,YUM,119.448334,130.552502,1041061,6842000000.0,1325000000.0,647000000.0,558000000.0,0.0,5846000000.0,11886000000.0,1665000000.0,1609000000.0,0.0,398000000.0
414,ZION,56.84,36.225833,109380,614000000.0,907000000.0,657000000.0,185000000.0,29000000.0,89545000000.0,651000000.0,0.0,0.0,4893000000.0,0.0


In [629]:
#Step 5
data_val = pd.merge(closes_val,cumulative_fundamentals_val,on='Symbol')
data_val = data_val[(data_val['Revenue'] != 0)&(data_val['Net Income'] != 0)]

#Step 6
ratios_val = calculate_ratios(data_val,year_max,year_min)

In [631]:
ratios_val

,Symbol,Revenue Increase,Cash Ratio,Current Ratio,Interest to Income,Debt to Income,Net income Margin,ROA,ROE,Debt-to-assets,Debt-to-equity,Hist Revenue Increase
0,MMM,0.757028,0.383808,1.542371,0.079779,0.334657,0.169184,0.124658,0.393357,0.343106,1.082665,0
1,ABT,0.943184,0.638001,1.628511,0.080485,0.324679,0.158821,0.093138,0.188982,0.225328,0.457204,0
3,ACN,1.018562,0.450243,1.233251,0.006881,0.001334,0.111653,0.145507,0.311098,0.001165,0.002491,1
5,ADBE,1.218171,0.521161,1.106791,0.023549,0.105130,0.270135,0.175078,0.338481,0.152770,0.295353,1
14,ALK,0.904825,0.082128,0.676608,1.862069,4.758621,0.006013,0.004089,0.015199,0.153250,0.569706,0
...,...,...,...,...,...,...,...,...,...,...,...,...
397,VMC,1.177532,0.168827,1.988292,0.293495,0.000867,0.078809,0.040500,0.083206,0.276924,0.568932,1
398,WMT,1.114414,0.169766,0.927797,0.120086,0.201076,0.024339,0.056930,0.167441,0.153831,0.452440,1
407,WHR,0.800053,0.330130,1.078064,-0.125745,-0.164130,-0.076607,-0.088239,-0.646832,0.444464,3.258134,0
409,WYNN,1.401426,2.088408,2.224542,-1.535628,-1.291814,-0.112823,-0.031595,0.564511,0.906663,-16.199197,1


# 7. Creating/Training Neural Network

In [288]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [643]:
# Removing Extreme Ouliers from datasets
ratio_clean = ratios[(np.abs(ratios.drop(['Symbol','Hist Revenue Increase'],axis=1) < 3).all(axis=1))]
ratio_clean_val = ratios_val[(np.abs(ratios_val.drop(['Symbol','Hist Revenue Increase'],axis=1) < 3).all(axis=1))]                                         

In [653]:
#Setting up Train Data
X_train = ratio_clean.drop(['Symbol','Hist Revenue Increase','Revenue Increase'],axis=1).values
y_train = ratio_clean['Hist Revenue Increase']

X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)

#Creating Model using Tensorflow
class Model(nn.Module):
    
    def __init__(self,in_features=9,h1=30,h2=40,h3=60,out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,h3)
        self.out = nn.Linear(h3,out_features)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        
        return x
    
model = Model()

#Training Model
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 201
losses = []
for i in range(epochs):
    y_pred = model.forward(X_train)

    loss = criterion(y_pred, y_train)
    
    losses.append(loss)
    
    if i % 50 == 0:
        print(f'Epoch: {i} and loss: {loss}')
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0 and loss: 1.1918352842330933
Epoch: 50 and loss: 0.5119813680648804
Epoch: 100 and loss: 0.183047354221344
Epoch: 150 and loss: 0.036611851304769516
Epoch: 200 and loss: 0.013807502575218678


# 9. Data Validation

In [654]:
X_test = ratio_clean_val.drop(['Symbol','Hist Revenue Increase','Revenue Increase'],axis=1).values
y_test = ratio_clean_val['Hist Revenue Increase']

X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

In [655]:
correct = 0
y_val = []
for i in range(len(X_test)):
    y_val = model.forward(X_test[i])

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
        correct +=1

print(f'Accuracy: {(correct/len(X_test))*100}%')

Accuracy: 60.17699115044248%
